In [52]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [53]:
x_train = pd.read_csv("../../human_train.csv").drop(["Activity","subject"],axis=1)
x_test = pd.read_csv("../../human_test.csv").drop(["Activity","subject"],axis=1)
y_train = pd.read_csv("../../human_train.csv",usecols=["Activity"])
y_test = pd.read_csv("../../human_test.csv",usecols=["Activity"])

In [54]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((7352, 561), (2947, 561), (7352, 1), (2947, 1))

In [55]:
# Initialize and train logistic regression model
log_reg = LogisticRegression(max_iter=1000)  # Increase max_iter if it doesn't converge
log_reg.fit(x_train, y_train)

# Make predictions on the test set
y_pred = log_reg.predict(x_test)

# Calculate and print accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy:", accuracy)

C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Test accuracy: 0.9613165931455717


#### <b><code>Remove Duplicate </code><b/>

In [56]:
x_train =x_train.T.drop_duplicates().T
x_test = x_test.T.drop_duplicates().T
x_train.shape,x_test.shape

((7352, 540), (2947, 540))

#### <b><code>Variance Threshold</code><b/>

In [57]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0.05)

In [58]:
sel.fit(x_train)
sel.fit(x_test)

VarianceThreshold(threshold=0.05)

In [59]:
columns = x_train.columns[sel.get_support()]
columns

Index(['tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z',
       'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z',
       'tBodyAcc-max()-X', 'tBodyAcc-max()-Y', 'tBodyAcc-max()-Z',
       'tBodyAcc-min()-X',
       ...
       'fBodyBodyGyroJerkMag-meanFreq()', 'fBodyBodyGyroJerkMag-skewness()',
       'fBodyBodyGyroJerkMag-kurtosis()', 'angle(tBodyAccMean,gravity)',
       'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)'],
      dtype='object', length=329)

In [60]:
x_train = sel.transform(x_train)
x_test = sel.transform(x_test)

In [61]:
x_train = pd.DataFrame(x_train,columns = columns)
x_test = pd.DataFrame(x_test,columns = columns)

In [62]:
x_train.shape,x_test.shape

((7352, 329), (2947, 329))

#### <b><code>Correlation</code><b/>

In [63]:
corr_matrix = x_train.corr()
columns = corr_matrix.columns

In [64]:
drop_col = []

for i in range (len(columns)):
    for j in range (i+1,len(columns)):
        if corr_matrix.loc[columns[i],columns[j]] > 0.95:
                drop_col.append(columns[j])

In [65]:
len(drop_col)

1457

In [66]:
drop_col = set(drop_col)
len(drop_col)

184

In [67]:
x_train.drop(drop_col,axis = 1,inplace=True)
x_test.drop(drop_col,axis = 1,inplace=True)

In [68]:
x_train.shape,x_test.shape

((7352, 145), (2947, 145))

#### <b><code>Anova</code></b>

In [69]:
from sklearn.feature_selection import f_classif,SelectKBest
sel = SelectKBest(f_classif,k=100).fit(x_train,y_train)

C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [70]:
columns = x_train.columns[sel.get_support()]
x_train = sel.transform(x_train)
x_test = sel.transform(x_test)

x_train = pd.DataFrame(x_train, columns=columns)
x_test = pd.DataFrame(x_test, columns=columns)

In [71]:
x_train.shape,x_test.shape

((7352, 100), (2947, 100))

In [72]:
# Initialize and train logistic regression model
log_reg = LogisticRegression(max_iter=1000)  # Increase max_iter if it doesn't converge
log_reg.fit(x_train, y_train)

# Make predictions on the test set
y_pred = log_reg.predict(x_test)

# Calculate and print accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy:", accuracy)

C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Test accuracy: 0.9317950458092976
